In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:92% !important; }</style>"))

%load_ext autoreload
%autoreload 2
import sys, os
from os.path import expanduser
## actions required!!!!!!!!!!!!!!!!!!!! change your folder path 
path = "~/Documents/G3/MA-prediction"
path = expanduser(path)
sys.path.append(path)

import data_science_MA_kit as dsk
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import wrds
import re

pd.options.mode.chained_assignment = None

In [2]:
import wrds
db = wrds.Connection()

Enter your WRDS username [yizhan]:olivershu
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
Loading library list...
Done


# Data processing 2: Merge with CRSP

In this notebook we merge the deals dataset from SDC with CRSP database. CRSP is the database for historical prices of securities especially equities. 

Specifically we will do the following:

- Look for the `permno` of targets and acquirors in CRSP database. `permno` is CRSP's own identifier for each equity.
- For completed deals, look for delisting code, dates and returns in CRSP database.

## Load data

In [3]:
# read hdf file
filepath = f"{path}/data/df_basic_cleaning.h5"
df = pd.read_hdf(filepath)

dsk.print_shape(df)
df.tail()

The dataset is of size (9854, 94).


,statc,one_day,aone_day,dao,da,dateannorig_days,de,dateeffexp,dw,definitive_agt,...,pricebook,eqvalcf,eqvalsales,eqval,tlia,cass,clia,lockup,dae,vest
master_deal_no,,,,,,,,,,,,,,,,,,,,,
3846599020,P,2021-12-14,2021-12-14,2021-12-15,2021-12-15,0,NaT,2022-12-31,NaT,Yes,...,6.389,16.389,4.750,5699.204,1265.1,535.0,205.9,No,No,No
3847933020,C,2021-12-16,NaT,2021-12-17,2021-12-17,0,2022-05-13,2022-06-30,NaT,Yes,...,5.573,59.340,5.089,2456.130,347.3,237.8,147.5,No,No,No
3848920020,C,2021-07-09,2021-12-17,2021-07-11,2021-12-19,161,2022-03-31,2022-04-01,NaT,Yes,...,1.346,13.529,0.621,784.682,400.0,433.0,201.0,No,No,No
3847595020,C,2021-12-15,2021-12-17,2021-12-16,2021-12-20,4,2022-06-08,2022-06-06,NaT,No,...,7.331,20.551,4.971,28373.205,3703.3,2440.2,1551.3,No,No,No
3851185020,C,NaT,2021-12-22,2021-12-23,2021-12-23,0,2022-05-27,2022-05-27,NaT,Yes,...,14.716,11.498,2.984,6082.266,2905.9,943.8,487.7,No,No,No


# Get target & acquiror permno
`permno` is the unique identifier for each security in CRSP database. We need to match each target and (public) acquiror from SDC with CRSP, by its ticker or cusip at the announcement day.

In [4]:
# process ticker and cusip
df.ttic = df.ttic.str.replace("'", "")
df.atic = df.atic.str.replace("'", "")
df.tcu = df.tcu.str.upper()
df.acu = df.acu.str.upper()

In [5]:
# will run 8-10 mins
columns = ['tpermno', 'ttic_CRSP', 'tcu_CRSP', 'tn_CRSP']
tpermno_match = dsk.ser_apply_function_args(df[['ttic', 'tcu', 'da']], 
                                              dsk.get_stock_permno_from_ser_CRSP,
                                               return_as_df=True,
                                               columns=columns,
                                               db=db, 
                                               return_id=True)

100%|███████████████████████████████████████| 9854/9854 [05:07<00:00, 32.00it/s]


In [6]:
columns = ['apermno', 'atic_CRSP', 'acu_CRSP', 'an_CRSP']
apermno_match = dsk.ser_apply_function_args(df[['atic', 'acu', 'da']], 
                                              dsk.get_stock_permno_from_ser_CRSP,
                                               return_as_df=True,
                                               columns=columns,
                                               db=db, 
                                               return_id=True)

100%|███████████████████████████████████████| 9854/9854 [05:53<00:00, 27.87it/s]


## save intermediate results

In [7]:
# filepath = f"{path}/data/df_tpermno.h5"
# df_tpermno_match.to_hdf(filepath, key='tpermno')

# filepath = f"{path}/data/df_apermno.h5"
# df_apermno_match.to_hdf(filepath, key='apermno')

# Get target delisting information

In [9]:
columns = ['delist_code', 'last_trade_date', 'delist_date', 'delist_amount', 'delist_return']
delist = dsk.ser_apply_function_args(tpermno_match.tpermno[df.statc.eq('C')], 
                                           dsk.get_delisting_information, 
                                           return_as_df=True, 
                                           columns = columns,
                                           db=db)

100%|███████████████████████████████████████| 7799/7799 [01:32<00:00, 84.19it/s]


# save intermediate results

In [10]:
pathfile = f"{path}/data/CRSP_results.h5"
# concat three files from CRSP database
CRSP_results = pd.concat([tpermno_match, apermno_match, delist], axis=1)
CRSP_results.to_hdf(pathfile, key = 'CRSP', mode='w')

/var/folders/l5/gqn1skjd0v366h5y3kbms6lh0000gn/T/ipykernel_81322/775729973.py:4: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['ttic_CRSP', 'tcu_CRSP', 'tn_CRSP', 'atic_CRSP', 'acu_CRSP', 'an_CRSP',
       'last_trade_date', 'delist_date'],
      dtype='object')]

  CRSP_results.to_hdf(pathfile, key = 'CRSP', mode='w')


In [11]:
# combine CRSP result with the dataset after basic cleaning
loc_names = ['ttic', 'ttic', 'tcu', 'tn'] + ['atic', 'atic', 'acu', 'an'] + ['att'] * 5
dsk.insert_cols(df, loc_names, CRSP_results.columns, CRSP_results)

pathfile = f"{path}/data/df_merge_CRSP.h5"
df.to_hdf(pathfile, key = 'df', mode='w')

/var/folders/l5/gqn1skjd0v366h5y3kbms6lh0000gn/T/ipykernel_81322/1230451088.py:6: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['statc', 'one_day', 'aone_day', 'dao', 'da', 'de', 'dateeffexp', 'dw',
       'definitive_agt', 'da_date', 'dateval', 'dcom', 'dcomeff',
       'last_trade_date', 'delist_date', 'att', 'attitude_change_yn',
       'initial_rec', 'tn_CRSP', 'tn', 'ttic_CRSP', 'ttic', 'tcu_CRSP', 'tcu',
       'texch', 'an_CRSP', 'an', 'apub', 'atic_CRSP', 'atic', 'acu_CRSP',
       'acu', 'anatc', 'aexch', 'cross', 'ttf_macro_desc', 'ttf_mid_desc',
       'atf_macro_desc', 'atf_mid_desc', 'valamend', 'consid_struct_desc',
       'consid', 'consido', 'consids', 'cha', 'tend', 'term', 'synop', 'hdate',
       'hosthprice', 'hval', 'hevent', 'hosthval', 'competecode', 'competeval',
       'lbo', 'afinancial', 'alp', 'aspv', 'awk', 'hedge_fund_invo